In [2]:
import cv2
import os

# augmentation libs
import albumentations as A
from albumentations.pytorch import ToTensorV2

# NN elements
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torch.nn as nn


# Data preparation

In [3]:
# Define folders with data
train_folder = 'D:/Enginering/CatsvsDogs/data/train/'
test_folder = 'D:/Enginering/CatsvsDogs/data/test/'

In [4]:
# train/test/valid split
train_images = [os.path.join(train_folder, img) for img in os.listdir(train_folder)]
test_images = [os.path.join(train_folder, img) for img in os.listdir(test_folder)]

val_images = train_images[21000:]
train_images = train_images[:21000]

## Dataset

In [5]:
# Dataset class
class CatsVsDogsDataset(Dataset):
    def __init__(self, images_filepaths, transform=None):
        self.images_filepaths = images_filepaths
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.images_filepaths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # labeling
        if 'cat' in os.path.normpath(train_images[0]).split(os.sep)[-1]:
            label = 1.0
        else:
            label = 0.0
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        return image, label

## Augmentation

In [6]:
train_transform = A.Compose(
    [
        A.SmallestMaxSize(max_size=160),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.RandomCrop(height=128, width=128),
        A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)
train_dataset = CatsVsDogsDataset(images_filepaths=train_images, transform=train_transform)

In [7]:
val_transform = A.Compose(
    [
        A.SmallestMaxSize(max_size=160),
        A.CenterCrop(height=128, width=128),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)
val_dataset = CatsVsDogsDataset(images_filepaths=val_images, transform=val_transform)

# Model

In [8]:
params = {
    "model": "resnet50",
    "device": "cpu",
    "lr": 0.001,
    "batch_size": 4,
    "num_workers": 0,
    "epochs": 10,
}

In [14]:
# model defenition
model = getattr(models, params["model"])(pretrained=False, num_classes=1,)
model = model.to(params["device"])
criterion = nn.BCEWithLogitsLoss().to(params["device"])
optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])

d:\Conda\envs\catdog\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
d:\Conda\envs\catdog\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [15]:
# DataLoaders defenition
train_loader = DataLoader(
    train_dataset, batch_size=params["batch_size"], shuffle=True, num_workers=params["num_workers"],
)
val_loader = DataLoader(
    val_dataset, batch_size=params["batch_size"], shuffle=False, num_workers=params["num_workers"],
)

In [16]:
def train(train_loader, model, criterion, optimizer):
    model.train()
    for images, target in train_loader:
        images = images
        target = target.float().view(-1, 1)
        output = model(images)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [17]:
def validate(val_loader, model, criterion):
    model.eval()
    with torch.no_grad():
        for images, target in val_loader:
            images = images
            target = target.float().view(-1, 1)
            output = model(images)
            loss = criterion(output, target)


In [18]:
for epoch in range(1, params["epochs"] + 1):
    print('EPOCH ' + str(epoch))
    train(train_loader, model, criterion, optimizer)
    validate(val_loader, model, criterion)

1


KeyboardInterrupt: 